In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import numpy as np
import socket, struct
import netaddr

import xxhash
%matplotlib inline

In [3]:
tp = pd.read_csv('capture20110818.pcap.netflow.labeled.csv', chunksize=1000)
df = pd.concat(tp, ignore_index=True)

# preprocessing
ips = df['SrcIPAddr:Port'].str.split(':')
df['SrcIPAddr'] = ips.str[0]
df['SrcIPPort'] = ips.str[-1]
ips = df['DstIPAddr:Port'].str.split(':')
df['DstIPAddr'] = ips.str[0]
df['DstIPPort'] = ips.str[-1]

df.head()

,Date,Flow_start,Durat,Prot,SrcIPAddr:Port,tmp,DstIPAddr:Port,Flags,Tos,Packets,Bytes,Flows,Label,Labels,SrcIPAddr,SrcIPPort,DstIPAddr,DstIPPort
0,2011-08-18,10:19:13.328,0.002,TCP,147.32.86.166:33426,->,212.24.150.110:25443,FRPA_,0,4,321,1,Background,NaN,147.32.86.166,33426,212.24.150.110,25443
1,2011-08-18,10:19:13.328,4.995,UDP,82.39.2.249:41915,->,147.32.84.59:43087,INT,0,617,40095,1,Background,NaN,82.39.2.249,41915,147.32.84.59,43087
2,2011-08-18,10:19:13.329,4.996,UDP,147.32.84.59:43087,->,82.39.2.249:41915,INT,0,1290,1909200,1,Background,NaN,147.32.84.59,43087,82.39.2.249,41915
3,2011-08-18,10:19:13.330,0.000,TCP,147.32.86.166:42020,->,147.32.192.34:993,A_,0,1,66,1,Background,NaN,147.32.86.166,42020,147.32.192.34,993
4,2011-08-18,10:19:13.330,0.000,TCP,212.24.150.110:25443,->,147.32.86.166:33426,FPA_,0,2,169,1,Background,NaN,212.24.150.110,25443,147.32.86.166,33426


In [4]:
confidence_interval = 0.99
error_rate = 0.001
w = int(np.ceil(2/error_rate)) #amount of 'counters' within a hash function
d = int(np.ceil(-np.log(1-confidence_interval)/np.log(2))) #amount of hash functions

print(w, d)


def update_wd(_hash,loc):
    wd[_hash][loc] = wd[_hash][loc] + 1

2000 7


In [5]:
wd = np.zeros((d, w))
#create the count min table
for i in range(d):
    #compute hashes of each source ip
    df['hash' + str(i)] = df['SrcIPAddr'].apply(lambda row: xxhash.xxh32(row, seed=i).intdigest())
    #update row in count min table for these hashes 
    df['hash' + str(i)].apply(lambda row: update_wd(i,row%w))
    
    
df.head()

,Date,Flow_start,Durat,Prot,SrcIPAddr:Port,tmp,DstIPAddr:Port,Flags,Tos,Packets,...,SrcIPPort,DstIPAddr,DstIPPort,hash0,hash1,hash2,hash3,hash4,hash5,hash6
0,2011-08-18,10:19:13.328,0.002,TCP,147.32.86.166:33426,->,212.24.150.110:25443,FRPA_,0,4,...,33426,212.24.150.110,25443,2505731307,2375472135,1602117669,1173467844,3145439405,4051052248,3152582910
1,2011-08-18,10:19:13.328,4.995,UDP,82.39.2.249:41915,->,147.32.84.59:43087,INT,0,617,...,41915,147.32.84.59,43087,626978567,2169557788,523794326,4103352933,3627694176,1210628020,629576023
2,2011-08-18,10:19:13.329,4.996,UDP,147.32.84.59:43087,->,82.39.2.249:41915,INT,0,1290,...,43087,82.39.2.249,41915,1115998567,1593929192,164951896,2222951432,2722666753,2485698111,180318254
3,2011-08-18,10:19:13.330,0.000,TCP,147.32.86.166:42020,->,147.32.192.34:993,A_,0,1,...,42020,147.32.192.34,993,2505731307,2375472135,1602117669,1173467844,3145439405,4051052248,3152582910
4,2011-08-18,10:19:13.330,0.000,TCP,212.24.150.110:25443,->,147.32.86.166:33426,FPA_,0,2,...,25443,147.32.86.166,33426,1720353717,4226395790,3408876200,3059175285,3535919388,3910918517,2485411514


In [6]:
print(wd)

[[3008. 2151.  206. ...  117.  173.  175.]
 [  79.   69.  996. ...  102.  276.   62.]
 [ 120.  263.   68. ...  158.   86.   96.]
 ...
 [ 135.   82.   66. ...   48. 2881.  127.]
 [ 142.  340.   95. ...  151. 7155.  250.]
 [  43.   65.  111. ...  293.  103.   83.]]


In [37]:
#for every unique ip calculate the hash row location and take the minimum value
df['freq'] = pd.Series(np.inf, index=df.index)

for i in range(d):
    df['freq'+str(i)] = df['hash'+str(i)].apply(lambda x: wd[i][x%w])
    df['freq'] = df[['freq','freq'+str(i)]].min(axis=1)

In [50]:
df = df.drop_duplicates('SrcIPAddr')
df = df.sort_values(['freq'], ascending=False)
df[['SrcIPAddr','freq']].head()

,SrcIPAddr,freq
24,147.32.84.229,179860.0
81,147.32.80.9,109348.0
2,147.32.84.59,101067.0
30,147.32.84.138,49471.0
9,147.32.84.118,12788.0
